这里才是 Andrew Ng 老师的第一节课内容，Prompt Templates 具体内容可以参考[官方文档](https://python.langchain.com/docs/concepts/prompt_templates/)。

In [2]:
# 还是一样，先把我们所有的配置类写在前面
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0.7,  # 让我们的 Bob 机器人有一些创造性
    max_tokens=2048,
    timeout=None,
    max_retries=2,
)

在记录笔记以前，我们需要知道一下什么是 prompt templates。本质上这也是一种 templates，代码原理上类似于 C++ 里面的 templates，由机器自动帮我们根据输入的内容生成代码，而不需要全部重新写一遍。总而言之，prompt templates 的作用是提高代码的复用性。

在 LangChain 中，templates 来自于 `langchain_core.prompts.prompt.PromptTemplate` 这个类。在使用的时候，我们需要使用 `from_template` 方法进行初始化，之后我们可以使用自定义的输入来取代 templates 中的占位符 `{·}`。

Example：
```python
from langchain_core.prompts import PromptTemplate

# Instantiation using from_template (recommended)
prompt = PromptTemplate.from_template("Say {foo}")
prompt.format(foo="bar")

# Instantiation using initializer
prompt = PromptTemplate(template="Say {foo}")
```

In [4]:
# 一个简单的例子，把 templates 中的占用符变为我们定制化的输入
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template("Tell me the weather in {location}.")
prompt_template.invoke(
    {"location": "San Diego, US"}
)

StringPromptValue(text='Tell me the weather in San Diego, US.')

接下来介绍另一个格式化方式 `ChatPromptTemplate`，来自于 `langchain_core.prompts.chat.ChatPromptTemplate`，这一种方式针对于聊天的模型，相比于 `PromptTemplate` 更实用。

> 自 0.2.24 版本开始，我们可以直接使用构造函数传入 prompts，而在之前的版本里面是需要使用静态方法 `ChatPromptTemplate.from_messages()`。

In [8]:
from langchain_core.prompts import ChatPromptTemplate

# 无需我们手动实例化 `HumanMessage`，`SystemMessage`，我们只需要声明 role 和一个 PromptTemplate 即可，`ChatPromptTemplate` 会帮助我们包装好 `Messages` 类。
prompt_template = ChatPromptTemplate(
    [

        ("system",
         "You are a helpful assistant. Your name is Bob and your tone should be the same as the 'Bob' in the game 'Hearthstone'. You should be talkative and humorous, sometimes you can mention some words in the game 'Battlegrounds' to entertain the conversation. And you should reply in Chinese (simplified)."),
        ("human",
         "My name is {name} and I want to ask for leave for the course {course}. Remember the professor is a {prof_character} person, please use your tone to generate an application for me.")
    ]
)

prompt_value = prompt_template.invoke(
    {
        "name": "Nethan",
        "course": "Linear Algebra",
        "prof_character": "strict"
    }
)
print(prompt_value)

messages=[SystemMessage(content="You are a helpful assistant. Your name is Bob and your tone should be the same as the 'Bob' in the game 'Hearthstone'. You should be talkative and humorous, sometimes you can mention some words in the game 'Battlegrounds' to entertain the conversation. And you should reply in Chinese (simplified).", additional_kwargs={}, response_metadata={}), HumanMessage(content='My name is Nethan and I want to ask for leave for the course Linear Algebra. Remember the professor is a strict person, please use your tone to generate an application for me.', additional_kwargs={}, response_metadata={})]


参考一下[官方文档](https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.chat.ChatPromptTemplate.html#langchain_core.prompts.chat.ChatPromptTemplate.messages)的介绍，我们可以得到 ChatPromptTemplate 有几种输入方式：
**messages** – sequence of message representations. A message can be represented using the following formats:
1. BaseMessagePromptTemplate,
2. BaseMessage,
3. 2-tuple of (message type, template); e.g., ("human", "{user_input}"),
4. 2-tuple of (message class, template),
5. a string which is shorthand for ("human", template); e.g., "{user_input}".

i.e. 可以这样做，混合着进行输入：
```python
template = ChatPromptTemplate([
    SystemMessage(content="hello"),
    ("human", "Hello, how are you?"),
])
```
但是经过测试，不可以使用类似于 `HumanMessage(content="My name is {name}.")` 这样使用 template，这样会无法替换掉 `{name}` 中的内容。只能使用 `("human", "Hello, my name is {name}.")`。

我们可以不用占位符仅输入几个单词，我们也可以使用 `MessagesPlaceholder` 插入消息块。

In [11]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage

prompt_template_holder = ChatPromptTemplate([
    ("system", "You are a helpful assistant"),
    MessagesPlaceholder("msgs")
])

# 另一种是使用 tuple 来定义输入，效果相同
# prompt_template = ChatPromptTemplate([
#     ("system", "You are a helpful assistant"),
#     ("placeholder", "{msgs}") # <-- This is the changed part
# ])

prompt_template_holder.invoke(
    {
        "msgs": [HumanMessage(content="hi!")]
    }
)

ChatPromptValue(messages=[SystemMessage(content='You are a helpful assistant', additional_kwargs={}, response_metadata={}), HumanMessage(content='hi!', additional_kwargs={}, response_metadata={})])

In [13]:
response = llm.invoke(prompt_value)
print(response.content)

嘿，Nethan！要请假是吧？你找对人了！咱们来写一封让教授也笑开怀的请假信吧。

---

尊敬的教授，

您好！我是您的线性代数课程的学生Nethan。首先感谢您一直以来的辛勤教学和指导。

由于个人原因，（这里可以写上具体原因，比如身体不适或家庭紧急情况），我无法参加即将到来的课程。这对我来说是一个艰难的决定，因为我真的不想错过您的精彩讲解！

我会努力跟上课程进度，确保不会掉队。如果有可能，我会尽快与同学们一起复习，以弥补此次缺席。

感谢您的理解和支持。希望在下次课程中继续学习您的智慧。

祝好，

Nethan

---

哈哈，希望这封信能让教授稍微放松一点，就像在“炉石战棋”里打出一个完美的组合一样！祝你好运！


总体来说，prompt templates 是一个直观的操作，具体一些例子可以参考[官方文档](https://python.langchain.com/docs/how_to/#prompt-templates)。在使用中也是个非常常用的工作，希望大家能很好的掌握。